<img src="Bilder/ost_logo.png" width="240" align="right"/>
<div style="text-align: left"> <b> Applied Neural Networks | FS 2025 </b><br>
<a href="mailto:christoph.wuersch@ost.ch"> © Christoph Würsch, François Chollet </a> </div>
<a href="https://www.ost.ch/de/forschung-und-dienstleistungen/technik-neu/systemtechnik/ice-institut-fuer-computational-engineering"> Eastern Switzerland University of Applied Sciences OST | ICE </a>

[![Run in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ChristophWuersch/AppliedNeuralNetworks/blob/main/ANN10/10.1-Erweiterte_Nutzung_rekurrenter_neuronaler_Netze_pl.ipynb)

In [ ]:
# für Ausführung auf Google Colab auskommentieren und installieren
!pip install -q -r https://raw.githubusercontent.com/ChristophWuersch/AppliedNeuralNetworks/main/requirements.txt

# 10. Erweiterte Nutzung rekurrenter neuronaler Netze (RNNs)

Dieses Notizbuch enthält die Codebeispiele aus Kapitel 6, Abschnitt 3 von [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python?a_aid=keras&a_bid=76564dff). 

In diesem Abschnitt werden wir drei ausgeklügeltere Verfahren zur Verbesserung der Leistung und der Verallgemeinerungsfähigkeit von RNNs betrachten. 
- Nach der Lektüre dieses Abschnitts werden Sie das Wichtigste über die Verwendung von RNNs wissen. Wir werden die drei Konzepte anhand der Aufgabe erörtern, die Temperatur vorherzusagen. 
- Zu diesem Zweck stehen Zeitreihen mit Datenpunkten zur Verfügung, die von auf einem Gebäudedach installierten Sensoren erfasst wurden, wie z.B. Temperatur, Luftdruck und Luftfeuchtigkeit. Diese Daten verwenden wir, um die Temperatur 24 Stunden nach Erfassung des letzten Datenpunkts vorherzusagen. Das stellt sich als ziemlich anspruchsvolle Aufgabe heraus, die viele der typischen Schwierigkeiten veranschaulicht, die bei der Verwendung von Zeitreihen auftreten.

Wir behandeln die folgenden Verfahren:
- **_Rekurrentes Dropout-Verfahren_** – Hierbei handelt es sich um ein spezielles integriertes Verfahren, Dropout einzusetzen, um eine Überanpassung in rekurrenten Layern zu verhindern.
- **_Hintereinanderschaltung rekurrenter Layer_** – Dieses Verfahren erhöht die Repräsentationsfähigkeit des NNs (auf Kosten höheren Rechenbedarfs).
- **_Bidirektionale rekurrente Layer_** – Bei diesem Verfahren werden einem RNN dieselben Informationen auf unterschiedliche Weise bereitgestellt. Dadurch wird zum einen die Korrektklassifizierungsrate erhöht, zum anderen wiegen Probleme mit verloren gegangenen Informationen weniger schwer.

## Temperaturvorhersage (Wetterdaten Jena)
Bislang haben wir nur eine Art sequenzieller Daten betrachtet, nämlich Texte, wie
die IMDb-Filmbewertungen oder die Reuters-Datensammlung. Sequenzielle Daten
spielen jedoch keineswegs nur bei der Verarbeitung von Sprache eine Rolle. Die
Beispiele in diesem Abschnitt verwenden eine Zeitreihe mit Wetterdaten, die von
der Wetterstation am *Max-Planck-Institut für Biogeochemie in Jena* aufgezeichnet
wurden. http://www.bgc-jena.mpg.de/wetter/.

- Die Datenmenge enthält *14 verschiedene Messgrössen* (wie Lufttemperatur, Luftdruck, Luftfeuchtigkeit, Windrichtung usw.), die über mehrere Jahre hinweg im *Zehnminutentakt* aufgezeichnet wurden. 
- Die ursprünglichen Daten reichen zurückbis 2003, aber dieses Beispiel ist auf den Zeitraum von 2009 bis 2016 beschränkt.
- Diese Datenmenge ist perfekt dafür geeignet, den Umgang mit numerischen Zeitreihen zu erlernen. 
- Wir werden ein Modell erstellen, das als Eingabe einige Daten aus der jüngsten Vergangenheit entgegennimmt (die Wetterdaten für ein paar Tage) und die Lufttemperatur für einen Zeitpunkt vorhersagt, der 24 Stunden in der Zukunft liegt.


Um die Daten herunerladen zu können, müssen Sie auf einer Windows-Umgebung erst noch `wget` installieren

In [ ]:
import os
import zipfile
import requests

os.makedirs("data", exist_ok=True)
url = "https://s3.amazonaws.com/keras-datasets/jena_climate_2009_2016.csv.zip"
zip_path = "data/jena_climate_2009_2016.csv.zip"
csv_path = "data/jena_climate_2009_2016.csv"

if not os.path.exists(csv_path):
    r = requests.get(url)
    with open(zip_path, "wb") as f:
        f.write(r.content)
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall("data")


Schauen wir uns die Daten an.

In [ ]:
import os

data_dir = "data/"
fname = os.path.join(data_dir, "jena_climate_2009_2016.csv")

f = open(fname)
data = f.read()
f.close()

lines = data.split("\n")
header = lines[0].split(",")
lines = lines[1:]
print(len(lines))


Die Ausgabe besagt, dass die Datei 420.551 Zeilen mit Daten enthält. Jede Zeile ist
ein Zeitschritt: die Aufzeichnung von Uhrzeit und Datum sowie 14 wetterbezogenen
Messwerten. Darüber hinaus enthält die Datei die folgenden Kopfzeilen:

In [ ]:
print(header)


Wir müssen noch die "Double Quotes" entfernen.


In [ ]:
new_header = []
for item in header:
    item = item.replace('"', "")
    print(item)
    new_header.append(item)


In [ ]:
header = new_header
new_header


Nun wandeln wir die 420'551 Zeilen in ein Numpy-Array um:

In [ ]:
import numpy as np

float_data = np.zeros((len(lines), len(header) - 1))
for i, line in enumerate(lines):
    values = [float(x) for x in line.split(",")[1:]]
    float_data[i, :] = values


Hier wird beispielhaft der zeitliche Verlauf der Temperatur im gesamten Erfassungszeitraum
(in Grad Celsius) ausgegeben. In diesem Diagramm
sind die jährlich periodisch schwankenden Temperaturen klar erkennbar.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Visualisierung der Temperaturdaten
sns.set_theme(style="whitegrid")  # Setze den Stil auf "whitegrid"
temp = float_data[:, 1]  # temperature (in degrees Celsius)
plt.figure(figsize=(12, 6))  # Größere Figur für bessere Lesbarkeit
sns.lineplot(x=range(len(temp)), y=temp, color="blue")
plt.ylabel("Temperature (°C)")
plt.xlabel("Time Steps")
plt.title("Temperature Over Time")
plt.show()


Der folgende Plot zeigt den Temperaturverlauf eines kürzeren Zeitraums (der ersten zehn Tage). Die Daten wurden im Zehnminutentakt erfasst, also ergeben sich 144 Messwerte pro Tag.

In [ ]:
sns.set_theme(style="whitegrid")  # Setze den Stil auf "whitegrid"
plt.figure(figsize=(12, 6))  # Größere Figur für bessere Lesbarkeit
sns.lineplot(x=range(1440), y=temp[:1440], color="blue")
plt.grid(True)
plt.ylabel("Temperature (°C)")
plt.xlabel("Time Steps")
plt.title("Temperature Over First 10 Days")
plt.show()


In diesem Diagramm sind die täglich periodisch schwankenden Temperaturen gut erkennbar, insbesondere in den letzten vier Tagen. Beachten Sie auch, dass der Erfassungszeitraum in einem ziemlich kalten Wintermonat liegen muss.

- Wenn Sie versuchen würden, die Durchschnittstemperatur des nächsten Monats anhand der Messwerte einiger vorangegangener Monate vorherzusagen, wäre die Aufgabe aufgrund der verlässlichen jährlichen Periodizität der Daten ziemlich einfach.
- Betrachtet man jedoch die Daten nur einiger weniger Tage, sieht der Temperaturverlauf viel chaotischer aus. Lässt sich diese Zeitreihe anhand der Daten einiger weniger Tage vorhersagen? Dieser Frage gehen wir jetzt auf den Grund.

## Daten vorbereiten

Die genaue Aufgabenstellung lautet folgendermassen: 

- Wenn die Daten von `lookback` zurückliegenden Zeitschritten (ein Zeitschritt ist zehn Minuten lang) vorliegen und die Messwerte aller `steps` Zeitschritte verwendet werden, lässt sich dann die Temperatur in delay Zeitschritten vorhersagen? 

Wir verwenden die folgenden Werte für die Parameter:
- `lookback = 720` – Wir nutzen die Beobachtungsdaten der letzten fünf Tage.
-  `steps = 6` – Wir nutzen einen Messwert pro Stunde.
-  `delay = 144` – Die Temperatur soll für einen Zeitpunkt vorhergesagt werden, der 24 Stunden in der Zukunft liegt.


Wir müssen zunächst zwei Dinge erledigen:

- Die Daten müssen in ein für ein NN geeignetes **numerisches Format** umgewandelt werden. Das ist nicht weiter schwer: Die Daten liegen bereits in *numerischer Form* vor, eine Vektorisierung ist also nicht erforderlich. 
- Die Werte der Zeitreihen der Datenmenge sind jedoch von *verschiedener Grössenordnung*. Die Temperatur liegt typischerweise zwischen –20 und +30 Grad, der Luftdruck hingegen besitzt (gemessen in Millibar) meist Werte, die bei ca. 1.000 liegen.  Die Zeitreihen müssen also **unabhängig voneinander normiert** werden, damit sie kleine Werte von vergleichbarer Grössenordnung enthalten.
- Wir benötigen einen **Python-Generator**, der ein Array der aktuellen Fliesskommadaten entgegennimmt und einen Stapel (`batch`) mit Daten der jüngsten Vergangenheit sowie einen Zielwert für die zukünftige Temperatur zurückgibt. Da die Samples in der Datenmenge hochgradig redundant sind (Sample `N` und Sample `N + 1` werden grösstenteils denselben Wert besitzen), wäre es nicht gerade effektiv, für sämtliche Samples Speicherplatz zu reservieren. Stattdessen erzeugen wir die Samples anhand der ursprünglichen Daten in Echtzeit.

Die Daten werden folgendermaßen vorverarbeitet: Von den einzelnen Werten
wird der Mittelwert der Zeitreihe subtrahiert. Anschließend wird das Ergebnis
durch die Standardabweichung dividiert. Wir nutzen die ersten 200.000 Zeitschritte
als Trainingsdaten, daher muss zur Berechnung des Mittelwerts und der
Standardabweichung nur dieser Teil der Daten verwendet werden.

In [ ]:
mean = float_data[:200000].mean(axis=0)
float_data -= mean
std = float_data[:200000].std(axis=0)
float_data /= std


In [ ]:
mean, std

Das folgende Listing enthält den Code für den **Datengenerator**. Er liefert ein Tupel `(samples, targets)` zurück.

`samples` ist ein Stapel von Eingabedaten, und `targets` ist das dazugehörige Array, das die Zielwerte für die Temperaturen enthält. Der
Generator nimmt die folgenden Argumente entgegen:
* `data` – Das ursprüngliche Array mit Fliesskommazahldaten, das bereits normiert wurde.
* `lookback` – Die Anzahl der zurückliegenden Zeitschritte, die für die Eingabe berücksichtigt werden sollen.
* `delay` – Die Anzahl der Zeitschritte bis zu dem Zeitpunkt, für den die Temperatur vorhergesagt werden soll.
* `min_index` und `max_index` – Indizes des Datenarrays, die beschränken, welche Zeitschritte verwendet werden dürfen. Sie dienen dazu, Teile der Daten für die Validierung und das Testen zurückzuhalten.
* `shuffle` – Gibt an, ob die Samples durchmischt oder in chronologischer Reihenfolge zurückgegeben werden sollen.
* `batch_size` – Die Anzahl der Samples pro Stapel.
* `step` – Der zeitliche Abstand (angegeben in Zeitschritten) der zu verwendenden Samples. Er wird auf 6 gesetzt, sodass der Datenmenge ein Datenpunkt pro Stunde entnommen wird.

In [ ]:
np.shape(float_data)

In [ ]:
def generator(
    data, lookback, delay, min_index, max_index, shuffle=False, batch_size=128, step=6
):
    if max_index is None:
        max_index = len(data) - delay - 1
    i = min_index + lookback
    while 1:
        if shuffle:
            rows = np.random.randint(min_index + lookback, max_index, size=batch_size)
        else:
            if i + batch_size >= max_index:
                i = min_index + lookback
            rows = np.arange(i, min(i + batch_size, max_index))
            i += len(rows)

        samples = np.zeros((len(rows), lookback // step, data.shape[-1]))
        targets = np.zeros((len(rows),))
        for j, row in enumerate(rows):
            indices = range(rows[j] - lookback, rows[j], step)
            samples[j] = data[indices]
            targets[j] = data[rows[j] + delay][1]
        yield samples, targets


Nun verwenden wir die abstrakte generator-Funktion zur Instanziierung von drei Generatoren: einen für das Training, einen für die Validierung und einen für das Testen. 
- Diese Generatoren berücksichtigen jeweils unterschiedliche zeitliche Abschnitte der ursprünglichen Daten. 
- Der *Trainingsgenerator* verwendet die ersten 200.000 Zeitschritte, der *Validierungsgenerator* die nachfolgenden 100.000 und der *Testgenerator* die verbleibenden.

In [ ]:
lookback = 1440  # 10 days
step = 6  # 10 minutes
delay = 144  # 24 hours

batch_size = 128

train_gen = generator(
    float_data,
    lookback=lookback,
    delay=delay,
    min_index=0,
    max_index=200000,
    shuffle=True,
    step=step,
    batch_size=batch_size,
)
val_gen = generator(
    float_data,
    lookback=lookback,
    delay=delay,
    min_index=200001,
    max_index=300000,
    step=step,
    batch_size=batch_size,
)


In [ ]:
test_gen = generator(
    float_data,
    lookback=lookback,
    delay=delay,
    min_index=300001,
    max_index=None,
    step=step,
    batch_size=batch_size,
)

# This is how many steps to draw from `val_gen`
# in order to see the whole validation set:
val_steps = (300000 - 200001 - lookback) // batch_size

# This is how many steps to draw from `test_gen`
# in order to see the whole test set:
test_steps = (len(float_data) - 300001 - lookback) // batch_size


## Eine vernünftige Abschätzung ohne Machine Learning

Bevor wir zur Vorhersage der Temperaturen ein Deep-Learning-Modell einsetzen, das wie eine Blackbox arbeitet, sollten wir einen einfachen Ansatz ausprobieren, der auf dem gesunden Menschenverstand beruht. 
- Er dient der **Überprüfung, ob das Modell vernünftig arbeitet**, und liefert eine **Abschätzung der Leistung**, die es zu schlagen gilt, um den Nutzen komplexerer Machine-Learning-Modelle zu demonstrieren. 
- Solche auf dem *gesunden Menschenverstand* beruhende Abschätzungen erweisen sich als nützlich, wenn Sie eine neue Aufgabe in Angriff nehmen, für die es (noch) keine bekannte Lösung gibt. 
- Ein klassisches Beispiel hierfür sind *unausgewogene Klassifizierungsaufgaben* (inbalanced classification), bei denen bestimmte Klassen sehr viel häufiger sind als andere. Wenn eine Datenmenge zu 90% aus Instanzen der Klasse A und nur zu 10% aus Instanzen der Klasse B besteht, sagt einem der gesunde Menschenverstand, immer »A« zu wählen, wenn ein neues Sample klassifiziert wird. Ein solcher Klassifizierer erreicht insgesamt eine Korrektklassifizierungsrate von 90%. Dieses Ergebnis gilt es zu schlagen, um zu demonstrieren, dass auf Machine Learning beruhende Ansätze überhaupt von Nutzen sind. 
- In manchen Fällen erweist es sich als erstaunlich schwierig, so einfache  Abschätzungen zu übertreffen.

Im vorliegenden Fall können wir davon ausgehen, dass die Temperaturzeitreihen stetige Werte enthalten (die morgige Temperatur wird ähnlich hoch wie die heutige sein) und dass diese Werte täglich periodisch schwanken. Ein auf dem gesunden Menschenverstand beruhender Ansatz würde also immer vorhersagen, dass die Temperatur in 24 Stunden der momentanen Temperatur entspricht.

Wir bewerten diesen Ansatz jetzt anhand der `mae`-Metrik (**Mean Absolute Error, mittlerer absoluter Fehler**):


Und hier ist eine Schleife zur Bewertung.

In [ ]:
def evaluate_naive_method():
    batch_maes = []
    for step in range(val_steps):
        samples, targets = next(val_gen)
        preds = samples[:, -1, 1]
        mae = np.mean(np.abs(preds - targets))
        batch_maes.append(mae)
    print(np.mean(batch_maes))


evaluate_naive_method()


Der Code errechnet einen Wert von `0.29`. 
- Da die Temperaturdaten normiert sind (zentriert um 0 und Standardabweichung 1), ist dieser Wert nicht unmittelbar aussagekräftig.
- Eine Umrechnung ergibt einen mittleren absoluten Fehler von 0.29 × `temperature_std`, also 2.57 Grad Celsius.
- Dieser mittlere absolute Fehler ist realativ gross. Nun geht es darum, das Wissen über Deep Learning zu nutzen, um ein besseres Ergebnis zu erzielen.

## Ein elementarer Machine‑Learning‑Ansatz mit Lightning

**Best Practice:** Wie beim gesundem Menschenverstand gilt auch hier: Erst einfache, nicht rechenaufwendige Modelle (z. B. kleine Fully‑connected NNs in PyTorch‑Lightning) ausprobieren, bevor man komplexe RNNs einsetzt – so kann man sicherstellen, dass zusätzliche Komplexität wirklich Vorteile bringt.

- **TimeseriesDataset**:  
  Ein Generator, der aus den Roh‑Zeitreihendaten für Training, Validierung und Test sequenzielle Paare `(Input, Ziel)` erstellt.  
  - `lookback`, `step`, `delay` und `batch_size` legen dabei Fenstergröße, Schrittweite und Vorhersagehorizont fest.

- **TimeseriesModel (LightningModule)**:  
  - `Flatten()` → `Linear(input_shape, 32)` → `ReLU()` → `Linear(32, 1)`  
  - Keine Aktivierung im letzten Dense‑Layer (typisch für Regression).  
  - Verlustfunktion: mittlerer absoluter Fehler (`nn.L1Loss()`).  
  - Optimierer: RMSprop.

- **Training**:  
  - Datensätze und DataLoader für Training/Validation/Test werden analog zum Originalansatz erzeugt.  
  - Mit `trainer = L.Trainer(max_epochs=10, accelerator="auto", devices=1)` und `trainer.fit(model, train_loader, val_loader)` trainieren wir 10 Epochen.


In [ ]:
import lightning as L
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader


# ---- Generator als Dataset ----
class TimeseriesDataset(Dataset):
    def __init__(
        self,
        data,
        lookback,
        delay,
        min_index,
        max_index,
        step=6,
        batch_size=128,
    ):
        self.data = data
        self.lookback = lookback
        self.delay = delay
        self.step = step
        self.batch_size = batch_size
        self.min_index = min_index + lookback
        self.max_index = max_index if max_index is not None else len(data) - delay
        self.indices = list(range(self.min_index, self.max_index))

    def __len__(self):
        return self.max_index - self.min_index

    def __getitem__(self, idx):
        i = self.indices[idx]
        sample = self.data[i - self.lookback : i : self.step]
        target = self.data[i + self.delay][1]  # Temperatur als Ziel (wie im Original)
        return torch.tensor(sample, dtype=torch.float32), torch.tensor(
            target, dtype=torch.float32
        )


In [ ]:
# ---- Lightning-Modell ----
class TimeseriesModel(L.LightningModule):
    def __init__(self, input_shape):
        super().__init__()
        self.model = nn.Sequential(
            nn.Flatten(), 
            nn.Linear(input_shape, 64), nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 32), nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(32, 1)
        )
        self.loss_fn = nn.L1Loss()

        # Listen zum Mitschreiben
        self.train_loss_epoch = []
        self.val_loss_epoch = []

        self._train_losses = []
        self._val_losses = []

    def forward(self, x):
        return self.model(x).squeeze()

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.loss_fn(y_hat, y)
        self._train_losses.append(loss.detach())
        self.log("train_loss", loss, prog_bar=True)
        return loss

    def on_train_epoch_end(self):
        avg_loss = torch.stack(self._train_losses).mean()
        self.train_loss_epoch.append(avg_loss.item())
        self._train_losses.clear()

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.loss_fn(y_hat, y)
        self._val_losses.append(loss.detach())
        self.log("val_loss", loss, prog_bar=True)
        return loss

    def on_validation_epoch_end(self):
        avg_loss = torch.stack(self._val_losses).mean()
        self.val_loss_epoch.append(avg_loss.item())
        self._val_losses.clear()

    def configure_optimizers(self):
        return torch.optim.RMSprop(self.parameters())


In [ ]:
# ---- Parameter wie im Original ----
lookback = 1440 #  # 10 days
step = 6 # 10 minutes
delay = 144 # 24 hours
batch_size = 128

# Input-Größe für Flatten Layer
input_shape = (lookback // step) * float_data.shape[-1]

# ---- Datasets und Dataloaders ----
train_dataset = TimeseriesDataset(
    float_data, lookback, delay, 0, 200000, step, batch_size
)
val_dataset = TimeseriesDataset(
    float_data, lookback, delay, 200001, 300000, step, batch_size
)
test_dataset = TimeseriesDataset(
    float_data, lookback, delay, 300001, None, step, batch_size
)

train_loader = DataLoader(train_dataset, batch_size=batch_size)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)


In [ ]:
#get first batch
for x, y in train_loader:
    print(x.shape)
    print(y.shape)
    break

In [ ]:
# ---- Training ----
model = TimeseriesModel(input_shape=input_shape)
trainer = L.Trainer(max_epochs=10, accelerator="auto", devices=1)
trainer.fit(model, train_loader, val_loader)


Schauen wir uns die Lernkurven für Test- und Validierungsdatenset an.

In [ ]:
# plotte die Trainings- und Validierungsverluste
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=(12, 6))
plt.plot(
    np.arange(len(model.train_loss_epoch)),
    model.train_loss_epoch, "o-",
    label="Trainingsverlust",
)
plt.plot(
    np.arange(len(model.val_loss_epoch)),
    model.val_loss_epoch,"o-",
    label="Validierungsverlust",
)
plt.xlabel("Epoch")
plt.ylabel("Verlust")
plt.title("Trainings- und Validierungsverluste")
plt.legend()
plt.grid(True)
plt.show()



### Abbildung: Verlauf von Trainings- und Validierungsverlust  
- **Blaue Linie (Trainingsverlust):** Fällt von ~1.5 auf ~0.75, zeigt stetiges Lernen auf den Trainingsdaten.  
- **Orange Linie (Validierungsverlust):** Starke Schwankungen zwischen 2.4 und 0.33 – nach anfänglichem Anstieg bis Epoche 1 fällt er bis Epoche 3, steigt dann wieder an und pendelt sich zuletzt um 0.4–0.7 ein.

---

## Verbesserte Interpretation

1. **Teilerfolg gegenüber dem Menschenverstand-Ansatz**  
   - In einigen Epochen (z. B. um Epoche 3 und am Ende) erreicht der Validierungsverlust Werte, die mit der einfachen, auf gesundem Menschenverstand beruhenden Schätzung vergleichbar sind.  
   - Allerdings kommen diese Tiefpunkte unzuverlässig und nur kurzfristig zustande – insgesamt übertrifft das einfache Modell hier oft den NN-Ansatz.

2. **Wert der Abschätzung**  
   - Die stabile Performance des Baseline‑Ansatzes unterstreicht, wie viel implizites Wissen in heuristischen Methoden steckt.  
   - Ohne diese Abschätzung hätte man leicht den Eindruck, das NN liefere “gute” Ergebnisse, obwohl es den einfachen Trick nicht konstant nutzt.

3. **Hypothesenraum vs. einfache Lösung**  
   - Der gewählte Hypothesenraum (zweischichtige Fully‑connected NNs) enthält zwar theoretisch auch die “Baseline‑Funktion” – praktisch findet der Optimierer diese jedoch nicht zuverlässig.  
   - Komplexere Räume machen einfache Lösungen oft schwer zugänglich, wenn der Lernalgorithmus nicht speziell darauf ausgerichtet ist, sie zu erkunden.



Sie werden sich nun vielleicht fragen: 
- Wenn es offenbar ein einfaches und gut funktionierendes Modell gibt (der auf dem gesunden Menschenverstand beruhende
Ansatz), warum findet das Modell ihn dann nicht und verbessert ihn? 
- Weil das Modell nicht dafür ausgelegt ist, beim Training nach dieser einfachen Lösung zu suchen. Der Raum der Modelle, in dem wir nach einer Lösung suchen, also der **Hypothesenraum**, enthält alle möglichen zweischichtigen NNs mit der definierten Konfiguration. 
- Diese NNs sind schon ziemlich kompliziert. Und wenn Sie in einem Raum komplizierter Modelle nach einer Lösung suchen, dann ist der einfache und gut funktionierende Ansatz womöglich gar nicht erlernbar, selbst wenn er rein technisch betrachtet Teil des Hypothesenraums ist. Hierbei handelt es sich um eine ziemlich bedeutende Einschränkung des Machine Learnings im Allgemeinen: 

Sofern in den Lernalgorithmus nicht fest einprogrammiert ist, nach einer bestimmten Art einfacher Modelle zu suchen, wird beim Erlernen der Parameter womöglich eine einfache Lösung für eine einfache Aufgabe nicht gefunden.

## Ein erstes RNN

Der erste vollständig verbundene Ansatz hat nicht besonders gut funktioniert, das soll aber nicht heissen, dass Machine Learning auf diese Aufgabe nicht anwendbar ist. Der letzte Ansatz hat anfangs die Dimensionalität der Zeitreihe verringert, und dadurch wurden die zeitlichen Informationen aus den Eingabedaten entfernt. 

Betrachten wir die Daten doch als das, was sie tatsächlich sind: eine Sequenz, in der *Kausalität und Reihenfolge von Bedeutung* sind.  
- Wir werden nun ein Modell zur Verarbeitung rekurrenter Sequenzen ausprobieren, das für sequenzielle Daten dieser Art massgeschneidert sein sollte, eben weil es sich im Gegensatz zum ersten Ansatz die zeitliche Reihenfolge der Datenpunkte zunutze macht.
- Anstelle des im letzten Abschnitt vorgestellten `LSTM`-Layers verwenden wir einen `GRU`-Layer (**Gated Recurrent Unit**, zu Deutsch etwa »geschlossene rekurrente Einheit «), die 2014 von **_Chung et al._** [1] entwickelt wurde.
- `GRU`-Layer basieren auf den gleichen Prinzipien wie `LSTM`-Layer, sind jedoch etwas **besser optimiert und daher weniger rechenaufwendig**, wenngleich ihre Repräsentationsfähigkeit oft nicht an diejenige von `LSTM`-Layern heranreicht. 
- Einen solchen **Kompromiss** zwischen Rechenaufwand und Repräsentationsfähigkeit muss man beim Machine Learning ständig eingehen.

[1] [Junyoung Chung et al., Empirical Evaluation of Gated Recurrent Neural Networks on Sequence
Modeling, Conference on Neural Information Processing Systems (2014)](https://arxiv.org/abs/1412.3555)

In [ ]:
import lightning as L
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt


# ---- Dataset ----
class TimeseriesDataset(Dataset):
    def __init__(self, data, lookback, delay, min_index, max_index, step=6):
        self.data = data
        self.lookback = lookback
        self.delay = delay
        self.step = step
        self.min_index = min_index + lookback
        self.max_index = max_index if max_index is not None else len(data) - delay
        self.indices = list(range(self.min_index, self.max_index))

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        i = self.indices[idx]
        sample = self.data[i - self.lookback : i : self.step]
        target = self.data[i + self.delay][1]  # Temperatur als Ziel
        return torch.tensor(sample, dtype=torch.float32), torch.tensor(
            target, dtype=torch.float32
        )


In [ ]:
# ---- Lightning-Modell ----
class GRUForecastingModel(L.LightningModule):
    def __init__(self, input_size):
        super().__init__()
        self.save_hyperparameters()

        self.gru = nn.GRU(input_size=input_size, hidden_size=32, batch_first=True)
        self.fc = nn.Linear(32, 1)
        self.loss_fn = nn.L1Loss()

        # Batch-Verlustsammler pro Epoche
        self.train_epoch_losses = []
        self.val_epoch_losses = []

        # Logging über Epochen hinweg
        self.train_losses = []
        self.val_losses = []

    def forward(self, x):
        output, _ = self.gru(x)
        output = self.fc(output[:, -1, :])  # Letzter Zeitschritt
        return output

    def on_train_epoch_start(self):
        self.train_epoch_losses.clear()

    def on_validation_epoch_start(self):
        self.val_epoch_losses.clear()

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.loss_fn(y_hat, y.unsqueeze(1))
        self.train_epoch_losses.append(loss.item())
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.loss_fn(y_hat, y.unsqueeze(1))
        self.val_epoch_losses.append(loss.item())
        return loss

    def on_train_epoch_end(self):
        avg_train_loss = sum(self.train_epoch_losses) / len(self.train_epoch_losses)
        self.train_losses.append(avg_train_loss)
        self.log("train_loss", avg_train_loss, prog_bar=True)

    def on_validation_epoch_end(self):
        avg_val_loss = sum(self.val_epoch_losses) / len(self.val_epoch_losses)
        self.val_losses.append(avg_val_loss)
        self.log("val_loss", avg_val_loss, prog_bar=True)

    def configure_optimizers(self):
        return torch.optim.RMSprop(self.parameters())


In [ ]:
# ---- Parameter ----
lookback = 1440
step = 6
delay = 144
batch_size = 128

# Input-Größe für GRU
input_size = float_data.shape[-1]

# ---- Datasets und Dataloaders ----
train_dataset = TimeseriesDataset(float_data, lookback, delay, 0, 200000, step)
val_dataset = TimeseriesDataset(float_data, lookback, delay, 200001, 300000, step)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# ---- Training ----
model = GRUForecastingModel(input_size=input_size)
trainer = L.Trainer(max_epochs=10, accelerator="auto")
trainer.fit(model, train_loader, val_loader)


In [ ]:
# ---- Plot der Verluste ----
plt.figure(figsize=(12, 6))
plt.plot(model.train_losses,"o-", label="Train Loss")
plt.plot(model.val_losses,"o-", label="Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss (MAE)")
plt.title("Training and Validation Loss")
plt.legend()
plt.grid(True)
plt.show()


## 📊 Trainings- und Validierungsverlauf – Kurzfassung

- 🔹 **Train Loss** sinkt zu Beginn, steigt aber ab Epoche 3–4 wieder an → **Overfitting** beginnt.
- 🔸 **Validation Loss** bleibt relativ konstant oder steigt leicht → **keine echte Generalisierung**.

### 📌 Fazit:
- **Overfitting ab Epoche 3–4**
- ✅ Einsatz von **Early Stopping** oder **Regularisierung** (z. B. Dropout, L2) empfohlen.



## Rekurrentes Dropout-Verfahren zum Verhindern einer Überanpassung

- Den Graphen für das Training und die Validierung ist zu entnehmen, dass es zu einer **Überanpassung (overfitting)** kommt: 
- Die Werte der Verlustfunktion für Trainings- und Validierungsdaten divergieren nach einigen wenigen Epochen beträchtlich.
- Das **klassische Dropout-Verfahren** zum Verhindern einer Überanpassung ist Ihnen bereits vertraut: Zufällig ausgewählte Einheiten der Eingabe werden auf null gesetzt, um durch glückliche Umstände entstandene, aber irrelevante Korrelationen in den Trainingsdaten aufzulösen, die dem Layer übergeben werden. 

Die korrekte Anwendung des **Dropout-Verfahrens auf ein rekurrentes NN** ist allerdings *nicht trivial*. Die Anwendung des Dropout-Verfahrens vor einem rekurrenten Layer erschwert das Lernen, anstatt zur Regularisierung beizutragen. 

2015 hat *Yarin Gal* [2] in seiner Doktorarbeit über Bayes‘sches Deep Learning eine geeignete Methode für die Anwendung des Dropout-Verfahrens in RNNs beschrieben: 
- Bei allen Zeitschritten sollte die gleiche Dropout-Maske (das Muster der Einheiten, die auf null gesetzt werden) verwendet werden, anstatt die Dropout-Maske bei jedem Zeitschritt zufällig zu ändern. 
- Zur Regularisierung der Repräsentationen, die von rekurrenten `GRU`- oder `LSTM`-Layern gebildet werden, sollte zudem eine zeitlich konstante Dropout-Maske auf die inneren rekurrenten Aktivierungen der Layer angewendet werden (eine sogenannte rekurrente Dropout-Maske). 
- Die Anwendung der gleichen Dropout-Maske bei allen Zeitschritten ermöglicht es, dass die erlernten Fehler zeitlich im NN weitergegeben werden. Eine jeweils zufällige Dropout-Maske würde dieses Fehlersignal unterbrechen und damit dem Lernvorgang schaden.

[2] [Yarin Gal, Uncertainty in Deep Learning (Doktorarbeit), 13. Oktober 2016](http://mlg.eng.cam.ac.uk/yarin/blog_2248.html)


## ⚡️ Verwendung von LSTM- und GRU-Schichten mit cuDNN in PyTorch Lightning

- Wenn du in PyTorch oder PyTorch Lightning eine `nn.LSTM`- oder `nn.GRU`-Schicht auf der **GPU mit Standardparametern** verwendest, nutzt PyTorch automatisch die **cuDNN-beschleunigte Implementierung**.
- Diese cuDNN-Kernel stammen von **NVIDIA** und bieten **sehr hohe Performance**, da sie speziell für die GPU optimiert wurden.
- Allerdings sind sie **weniger flexibel**: Sobald du von den unterstützten Features abweichst, **fällt PyTorch automatisch auf eine langsamere (nicht-cuDNN) Implementierung zurück**.
- 🔥 **Wichtiges Beispiel**:  
  - **Recurrent Dropout** (d. h. Dropout zwischen Rechenschritten innerhalb der Sequenz) wird **nicht vom cuDNN-Kernel unterstützt**.
  - Wenn du z. B. `dropout > 0` in einem LSTM oder GRU mit `num_layers=1` setzt, wird die **schnelle cuDNN-Implementierung deaktiviert**.
  - Die Ausführung kann dann **2–5 mal langsamer** sein – besonders relevant bei größeren Sequenzen oder Batchgrößen.

### ✅ Empfehlung:
- Nutze `dropout` nur bei `num_layers > 1`, da PyTorch dann Dropout **zwischen den Schichten** einfügt, was von cuDNN unterstützt wird.
- Vermeide rekurrentes Dropout **innerhalb** der LSTM-/GRU-Zellen, wenn dir Performance wichtig ist.


In [ ]:
import lightning as L
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import numpy as np


# ---- Lightning-Modell ----
class GRUForecastingModel(L.LightningModule):
    def __init__(self, input_size):
        super().__init__()
        self.save_hyperparameters()

        self.gru = nn.GRU(
            input_size=input_size,
            hidden_size=128,
            batch_first=True,
            dropout=0.2,
            num_layers=2,
        )
        self.fc = nn.Linear(128, 1)
        self.loss_fn = nn.L1Loss()

        self.train_losses = []  # Liste für Plot
        self.val_losses = []

        self._train_epoch_losses = []  # temporäre Sammler
        self._val_epoch_losses = []

    def forward(self, x):
        output, _ = self.gru(x)
        return self.fc(output[:, -1, :])

    def on_train_epoch_start(self):
        self._train_epoch_losses.clear()

    def on_validation_epoch_start(self):
        self._val_epoch_losses.clear()

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.loss_fn(y_hat, y.unsqueeze(1))
        self._train_epoch_losses.append(loss.detach())
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.loss_fn(y_hat, y.unsqueeze(1))
        self._val_epoch_losses.append(loss.detach())
        return loss

    def on_train_epoch_end(self):
        avg_loss = torch.stack(self._train_epoch_losses).mean()
        self.train_losses.append(avg_loss.item())
        self.log("train_loss_epoch", avg_loss, prog_bar=True)  # Logging Epochenmittel

    def on_validation_epoch_end(self):
        avg_loss = torch.stack(self._val_epoch_losses).mean()
        self.val_losses.append(avg_loss.item())
        self.log("val_loss_epoch", avg_loss, prog_bar=True)  # Logging Epochenmittel

    def configure_optimizers(self):
        return torch.optim.RMSprop(self.parameters())


In [ ]:
# ---- Parameter ----
lookback = 1440
step = 6
delay = 144
batch_size = 128

# Input-Größe für GRU
input_size = float_data.shape[-1]

# ---- Datasets und Dataloaders ----
train_dataset = TimeseriesDataset(float_data, lookback, delay, 0, 200000, step)
val_dataset = TimeseriesDataset(float_data, lookback, delay, 200001, 300000, step)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# ---- Training ----
model = GRUForecastingModel(input_size=input_size)
trainer = L.Trainer(max_epochs=10, accelerator="auto")
trainer.fit(model, train_loader, val_loader)


In [ ]:
# ---- Plot der Verluste ----
plt.figure(figsize=(12, 6))
plt.plot(model.train_losses, "o-", label="Training Loss")
plt.plot(model.val_losses, "o-", label="Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss (MAE)")
plt.title("Training and Validation Loss")
plt.legend()
plt.grid(True)
plt.show()


## Hintereinanderschaltung rekurrenter Layer (stacking RNN layers)

Es gibt zwar keine Überanpassung mehr, allerdings sind wir offenbar auf einen Leistungsengpass gestossen, daher sollten wir in Betracht ziehen, die **Kapazität des NNs zu erhöhen**. 
Rufen Sie sich die Beschreibung des allgemeinen Machine-Learning-Workflows ins Gedächtnis: 
- Üblicherweise ist es sinnvoll, die Kapazität des NNs zu erhöhen, bis eine Überanpassung einsetzt und zum grössten Hindernis wird. (Vorausgesetzt, Sie haben die elementaren Schritte zum Abschwächen der Überanpassung, wie z.B. den Einsatz des Dropout-Verfahrens, bereits unternommen.)
- Solange es nicht zu einer allzu grossen Überanpassung kommt, können Sie die Kapazität wahrscheinlich noch erhöhen.

Zwecks Erhöhung der Kapazität eines NNs erhöht man typischerweise die Anzahl
der Einheiten in den Layern oder fügt zusätzliche Layer hinzu. 
- Die **Hintereinanderschaltung rekurrenter Layer ist ein klassisches Verfahren, leistungsfähigere
rekurrente NNs zu erstellen**. 
- Googles Übersetzungsalgorithmus beispielsweise verwendet eine Hintereinanderschaltung von sieben LSTM-Layern – das ist schon
enorm.
- Bei der Hintereinanderschaltung rekurrenter Layer sollten alle zwischenliegenden Layer ihre vollständige Ausgabesequenz (einen 3-D-Tensor) zurückliefern, nicht nur die Ausgabe des letzten Zeitschritts. 
- Zu diesem Zweck verwendet man das Argument `return_sequence=True`. 

In [ ]:
import lightning as L
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset


# ---- Lightning-Modell ----
class GRUForecastingModel(L.LightningModule):
    def __init__(self, input_size):
        super().__init__()
        self.save_hyperparameters()

        self.gru1 = nn.GRU(
            input_size=input_size, hidden_size=32, batch_first=True, dropout=0.1
        )
        self.gru2 = nn.GRU(input_size=32, hidden_size=64, batch_first=True, dropout=0.1)
        self.fc = nn.Linear(64, 1)
        self.loss_fn = nn.L1Loss()

        # Initialize attributes for tracking losses
        self._train_epoch_losses = []
        self._val_epoch_losses = []
        self.train_losses = []
        self.val_losses = []

    def forward(self, x):
        x, _ = self.gru1(x)
        x, _ = self.gru2(x)
        output = self.fc(x[:, -1, :])  # Letzter Zeitschritt
        return output

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.loss_fn(y_hat, y.unsqueeze(1))
        self._train_epoch_losses.append(loss)  # Append loss to track it
        self.log("train_loss", loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.loss_fn(y_hat, y.unsqueeze(1))
        self._val_epoch_losses.append(loss)  # Append loss to track it
        self.log("val_loss", loss, prog_bar=True)
        return loss

    def on_train_epoch_end(self):
        avg_loss = torch.stack(self._train_epoch_losses).mean()
        self.train_losses.append(avg_loss.item())
        self.log("train_loss_epoch", avg_loss, prog_bar=True)  # Logging Epochenmittel

    def on_validation_epoch_end(self):
        avg_loss = torch.stack(self._val_epoch_losses).mean()
        self.val_losses.append(avg_loss.item())
        self.log("val_loss_epoch", avg_loss, prog_bar=True)  # Logging Epochenmittel

    def configure_optimizers(self):
        return torch.optim.RMSprop(self.parameters())


In [ ]:
# ---- Parameter ----
lookback = 1440
step = 6
delay = 144
batch_size = 128

# Input-Größe für GRU
input_size = float_data.shape[-1]

# ---- Datasets und Dataloaders ----
train_dataset = TimeseriesDataset(float_data, lookback, delay, 0, 200000, step)
val_dataset = TimeseriesDataset(float_data, lookback, delay, 200001, 300000, step)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# ---- Training ----
model = GRUForecastingModel(input_size=input_size)
trainer = L.Trainer(max_epochs=10, accelerator="auto")
trainer.fit(model, train_loader, val_loader)


In [ ]:
# ---- Plot der Verluste ----
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=(12, 6))
plt.plot(model.train_losses, "o-", label="Training Loss")
plt.plot(model.val_losses, "o-", label="Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss (MAE)")
plt.title("Training and Validation Loss")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import lightning as L
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt


# ---- Lightning-Modell ----
class LSTMForecastingModel(L.LightningModule):
    def __init__(self, input_size):
        super().__init__()
        self.save_hyperparameters()

        self.lstm1 = nn.LSTM(input_size=input_size, hidden_size=32, batch_first=True)
        self.lstm2 = nn.LSTM(input_size=32, hidden_size=64, batch_first=True)
        self.fc = nn.Linear(64, 1)
        self.loss_fn = nn.L1Loss()

        # Listen zum Speichern der Verluste
        self.train_losses = []
        self.val_losses = []
        self._train_epoch_losses = []
        self._val_epoch_losses = []

    def forward(self, x):
        x, _ = self.lstm1(x)
        x, _ = self.lstm2(x)
        output = self.fc(x[:, -1, :])  # Letzter Zeitschritt
        return output

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.loss_fn(y_hat, y.unsqueeze(1))
        self._train_epoch_losses.append(loss)  # Append batch loss
        self.log("train_loss", loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.loss_fn(y_hat, y.unsqueeze(1))
        self._val_epoch_losses.append(loss)  # Append batch loss
        self.log("val_loss", loss, prog_bar=True)
        return loss

    def on_train_epoch_end(self):
        avg_loss = torch.stack(self._train_epoch_losses).mean()
        self.train_losses.append(avg_loss.item())
        self.log("train_loss_epoch", avg_loss, prog_bar=True)  # Logging Epochenmittel

    def on_validation_epoch_end(self):
        avg_loss = torch.stack(self._val_epoch_losses).mean()
        self.val_losses.append(avg_loss.item())
        self.log("val_loss_epoch", avg_loss, prog_bar=True)  # Logging Epochenmittel

    def configure_optimizers(self):
        return torch.optim.RMSprop(self.parameters())


In [ ]:
# ---- Parameter ----
lookback = 1440
step = 6
delay = 144
batch_size = 128

# Input-Größe für LSTM
input_size = float_data.shape[-1]

# ---- Datasets und Dataloaders ----
train_dataset = TimeseriesDataset(float_data, lookback, delay, 0, 200000, step)
val_dataset = TimeseriesDataset(float_data, lookback, delay, 200001, 300000, step)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# ---- Training ----
model = LSTMForecastingModel(input_size=input_size)
trainer = L.Trainer(max_epochs=10, accelerator="auto")
trainer.fit(model, train_loader, val_loader)


In [ ]:
# ---- Plot der Verluste ----
plt.figure(figsize=(12, 6))
plt.plot(model.train_losses,"o-", label="Train Loss")
plt.plot(model.val_losses,"o-", label="Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss (MAE)")
plt.title("Training and Validation Loss")
plt.legend()
plt.grid(True)
plt.show()


## Bidirektionale RNNs

Das letzte Verfahren, das in diesem Abschnitt vorgestellt wird, sind *bidirektionale RNNs*. Dabei handelt es sich um eine gebräuchliche Variante eines RNNs, mit der sich bei bestimmten Aufgaben eine bessere Leistung erzielen lässt als mit herkömmlichen RNNs. 
- Sie wird häufig zur Verarbeitung natürlicher Sprache eingesetzt (NLP) – man könnte sie als das »Schweizer Taschenmesser« des Deep Learnings für die Verarbeitung natürlicher Sprache bezeichnen.
- RNNs sind *von der Reihenfolge der Eingabe abhängig*: Sie verarbeiten die Zeitschritte der Eingabesequenzen der Reihe nach, und eine Durchmischung oder Umkehr der Zeitschritte kann die Repräsentationen, die das RNN aus der Sequenz extrahiert, völlig verändern. Aus genau diesem Grund sind RNNs besonders gut für Aufgaben geeignet, bei denen die Reihenfolge von Bedeutung ist, wie etwa bei der Temperaturvorhersage. 
- Ein **bidirektionales RNN** macht sich die Tatsache zunutze, dass RNNs empfindlich auf die Reihenfolge reagieren:
    * Es besteht aus *zwei herkömmlichen RNNs*, wie den bereits bekannten `GRU`- oder `LSTM`-Layern, die die Eingabesequenz in chronologischer bzw. umgekehrt chronologischer Reihenfolge verarbeiten und anschliessend ihre Repräsentationen verschmelzen.
    * Dank der Verarbeitung der Sequenz in beide Richtungen kann ein bidirektionales RNN Muster erfassen, die einem unidirektionalen RNN womöglich entgehen.

In [ ]:
class BidirectionalLSTMForecastingModel(L.LightningModule):
    def __init__(self, input_size):
        super().__init__()
        self.save_hyperparameters()

        self.lstm1 = nn.LSTM(
            input_size=input_size, hidden_size=32, batch_first=True, bidirectional=True
        )
        self.lstm2 = nn.LSTM(
            input_size=32 * 2, hidden_size=64, batch_first=True, bidirectional=True
        )
        self.fc = nn.Linear(64 * 2, 1)
        self.loss_fn = nn.L1Loss()

        # Listen zum Speichern der Verluste
        self.train_losses = []
        self.val_losses = []
        self._train_epoch_losses = []
        self._val_epoch_losses = []

    def forward(self, x):
        x, _ = self.lstm1(x)
        x, _ = self.lstm2(x)
        output = self.fc(x[:, -1, :])  # Letzter Zeitschritt
        return output

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.loss_fn(y_hat, y.unsqueeze(1))
        self._train_epoch_losses.append(loss)  # Append batch loss
        self.log("train_loss", loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.loss_fn(y_hat, y.unsqueeze(1))
        self._val_epoch_losses.append(loss)  # Append batch loss
        self.log("val_loss", loss, prog_bar=True)
        return loss

    def on_train_epoch_end(self):
        avg_loss = torch.stack(self._train_epoch_losses).mean()
        self.train_losses.append(avg_loss.item())
        self.log("train_loss_epoch", avg_loss, prog_bar=True)  # Logging Epochenmittel

    def on_validation_epoch_end(self):
        avg_loss = torch.stack(self._val_epoch_losses).mean()
        self.val_losses.append(avg_loss.item())
        self.log("val_loss_epoch", avg_loss, prog_bar=True)  # Logging Epochenmittel

    def configure_optimizers(self):
        return torch.optim.RMSprop(self.parameters())


In [ ]:
# ---- Parameter ----
lookback = 1440
step = 6
delay = 144
batch_size = 128

# Input-Größe für LSTM
input_size = float_data.shape[-1]

# ---- Datasets und Dataloaders ----
train_dataset = TimeseriesDataset(float_data, lookback, delay, 0, 200000, step)
val_dataset = TimeseriesDataset(float_data, lookback, delay, 200001, 300000, step)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# ---- Training ----
model = BidirectionalLSTMForecastingModel(input_size=input_size)
trainer = L.Trainer(max_epochs=10, accelerator="auto")
trainer.fit(model, train_loader, val_loader)


In [ ]:
# ---- Plot der Verluste ----
plt.figure(figsize=(12, 6))
plt.plot(model.train_losses,"o-", label="Train Loss")
plt.plot(model.val_losses,"o-", label="Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss (MAE)")
plt.title("Training and Validation Loss")
plt.legend()
plt.grid(True)
plt.show()


Die Tatsache, dass die RNN-Layer in den letzten Abschnitten die Sequenzen in chronologischer Reihenfolge verarbeitet haben (die ältesten Zeitschritte zuerst), ist bemerkenswerterweise eine vollkommen willkürliche Entscheidung – oder zumindest
eine Entscheidung, die wir bislang nicht infrage gestellt haben. 
- Wären die RNNs leistungsfähig genug gewesen, wenn sie die Eingabesequenzen beispielsweise **in umgekehrt chronologischer Reihenfolge (neuere Zeitschritte zuerst)** verarbeitet hätten? 

Das probieren wir nun aus und betrachten, was geschieht.  Wir benötigen lediglich eine Variante des Datengenerators, die die Eingabesequenzen entlang der Zeitdimension umkehrt. Ersetzen Sie die letzte Zeile einfach durch:

In [ ]:
import lightning as L
from lightning.pytorch.loggers import CSVLogger
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
import numpy as np


# ---- Fully Torch-native Reversed Time Axis Dataset ----
class ReversedTimeseriesDataset(Dataset):
    def __init__(self, data, lookback, delay, min_index, max_index, step=6):
        self.data = data
        self.lookback = lookback
        self.delay = delay
        self.step = step
        self.min_index = min_index + lookback
        self.max_index = min(
            max_index if max_index is not None else len(data), len(data) - delay
        )
        self.indices = list(range(self.min_index, self.max_index))

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        i = self.indices[idx]
        sample_np = self.data[i - self.lookback : i : self.step]  # still NumPy here
        sample = torch.from_numpy(sample_np).float()  # convert to torch tensor
        sample = torch.flip(sample, dims=[0])  # flip along time axis (dim 0)

        target = torch.tensor(self.data[i + self.delay][1], dtype=torch.float32)
        return sample, target



In [ ]:
# ---- Lightning-Modell ----
class GRUForecastingModel(L.LightningModule):
    def __init__(self, input_size):
        super().__init__()
        self.save_hyperparameters()  # loggt Hyperparameter automatisch
        self.gru = nn.RNN(input_size=input_size, hidden_size=32, batch_first=True)
        self.fc = nn.Linear(32, 1)
        self.loss_fn = nn.L1Loss()

        self.train_losses = []
        self.val_losses = []

    def forward(self, x):
        output, _ = self.gru(x)
        return self.fc(output[:, -1, :])  # Letzter Zeitschritt

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.loss_fn(y_hat, y.unsqueeze(1))
        self.train_losses.append(loss.detach())
        return loss

    def on_train_epoch_end(self):
        mean_loss = torch.stack(self.train_losses).mean()
        self.log("train_loss", mean_loss, prog_bar=True)
        self.train_losses.clear()

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.loss_fn(y_hat, y.unsqueeze(1))
        self.val_losses.append(loss.detach())
        return loss

    def on_validation_epoch_end(self):
        mean_loss = torch.stack(self.val_losses).mean()
        self.log("val_loss", mean_loss, prog_bar=True)
        self.val_losses.clear()

    def configure_optimizers(self):
        return torch.optim.RMSprop(self.parameters(), lr=0.01)


In [ ]:
# ---- Parameter ----
lookback = 1440  # 10 Tage bei Daten alle 10 Min
step = 6  # jede Stunde ein Wert
delay = 144  # Vorhersage 1 Tag in die Zukunft
batch_size = 128


# ---- Dataset / DataLoader ----
input_size = float_data.shape[-1]
train_dataset = TimeseriesDataset(float_data, lookback, delay, 0, 200000, step)
val_dataset = TimeseriesDataset(float_data, lookback, delay, 200001, 300000, step)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# ---- Logger ----
csv_logger = CSVLogger("logs", name="gru_forecasting")

# ---- Training ----
model = GRUForecastingModel(input_size=input_size)
trainer = L.Trainer(
    max_epochs=10,
    accelerator="auto",
    logger=csv_logger,
)

trainer.fit(model, train_loader, val_loader)


In [ ]:
# read csv logs/gru_forecasting/version_0/metrics.csv
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np

# Pfad zur CSV-Datei
csv_path = "logs/gru_forecasting/version_0/metrics.csv"
df = pd.read_csv(csv_path)
# Filtere nur Zeilen mit train_loss bzw. val_loss
train_df = df[["epoch", "train_loss"]].dropna().drop_duplicates(subset="epoch")
val_df = df[["epoch", "val_loss"]].dropna().drop_duplicates(subset="epoch")

# Füge sie auf Basis der Epoche zusammen
merged_df = pd.merge(train_df, val_df, on="epoch", how="outer").sort_values("epoch")

# Optional: Index zurücksetzen
merged_df.reset_index(drop=True, inplace=True)

# Ergebnis anzeigen
print(merged_df)

# Plot der Verluste
plt.figure(figsize=(12, 6))
plt.plot(merged_df["epoch"], merged_df["train_loss"],"o-", label="Train Loss")
plt.plot(merged_df["epoch"], merged_df["val_loss"],"o-", label="Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss (MAE)")
plt.title("Training and Validation Loss")
plt.legend()
plt.grid()
plt.show()


**Schlussfolgerung**:

- Das mit Sequenzen in umgekehrt chronologischer Reihenfolge trainierte GRUModell funktioniert sogar noch schlechter als der auf dem gesunden Menschenverstand beruhende Ansatz, was in diesem Fall darauf hinweist, dass die chronologische Verarbeitung für das Funktionieren des Modells wichtig ist.
- Das ergibt auch durchaus Sinn: Der zugrunde liegende `GRU`-Layer wird sich typischerweise besser an die jüngere als an die weiter zurückliegende Vergangenheit »erinnern«, und dementsprechend besitzen jüngere Wetterdaten für die Aufgabe eine grössere Aussagekraft als ältere. Aus diesem Grund ist der auf dem gesunden Menschenverstand beruhende Ansatz ziemlich leistungsfähig.
- Deshalb übertrifft die chronologische Version des Layers den mit Sequenzen in umgekehrt chronologischer Reihenfolge trainierten Layer bei Weitem. 

Beachten Sie jedoch, dass dies für viele andere Aufgaben, auch für die **Verarbeitung natürlicher Sprache, nicht zutrifft**:
- Anschaulich gesagt, hängt die Bedeutung eines Worts für das Verständnis eines  Satzes für gewöhnlich nicht davon ab, an welcher Stelle im Satz sich das Wort befindet. 


## Wrapping up

Nehmen Sie Folgendes aus diesem Abschnitt mit:
1. Es ist sinnvoll, eine **auf dem gesunden Menschenverstand beruhende Abschätzung vorzunehmen**, mit der Sie Ihre Ergebnisse vergleichen können. Wenn Sie nicht wissen, welche Leistung es zu schlagen gilt, können Sie auch nicht feststellen, ob Sie Fortschritte machen.
2. **Bottom-up**: Probieren Sie zunächst statt rechenaufwendiger Modelle einfache aus, um gegebenenfalls weiteren Aufwand zu rechtfertigen. Mitunter stellt sich heraus, dass ein einfaches Modell die beste Lösung ist.
3. Wenn Sie Daten verwenden, bei denen die **zeitliche Reihenfolge von Bedeutung** ist, sind **RNNs** bestens geeignet und den Modellen deutlich überlegen, die zunächst die Dimensionalität der zeitlichen Daten verringern.
4. Wenn Sie das **Dropout-Verfahren auf RNNs** anwenden, sollten Sie eine Dropout-Maske und eine rekurrente Dropout-Maske verwenden, die zeitlich konstant sind. Diese Masken sind in  rekurrenten Layern integriert, Sie brauchen also nur noch die Argumente `dropout` und `recurrent_dropout` anzugeben.
5. **Hintereinandergeschaltete RNNs bieten eine höhere Repräsentationsfähigkeit** als einzelne RNN-Layer. Sie sind allerdings auch erheblich rechenaufwendiger, deshalb lohnt ihr Einsatz nicht immer. Sie bieten bei komplexen Aufgaben (z.B. der maschinellen Übersetzung von Fremdsprachen) zwar deutliche Vorteile, die bei kleineren, einfachen Aufgaben aber nicht ins Gewicht fallen.
6. Bidirektionale RNNs, die Sequenzen in normaler und in umgekehrter Reihenfolge verarbeiten, sind gut für die Verarbeitung natürlicher Sprache geeignet, weisen jedoch bei sequenziellen Daten Schwächen auf, wenn die jüngste Vergangenheit erheblich informativer ist als der Anfang der Sequenz.

## Weiterfürhende Konzepte: Attention und Sequence Masking

Es gibt zwei bedeutende Konzepte, auf die wir an dieser Stelle nicht näher eingehen:
- **Recurrent Attention: rekurrente Berücksichtigung** und die 
- **Sequence Masking: Maskierung von Sequenzen**. 

Die beiden Konzepte sind insbesondere für die Verarbeitung natürlicher Sprache von Bedeutung, für die Temperaturvorhersage
jedoch kaum geeignet. Bei Interesse müssen Sie sich gegebenenfalls jenseits dieses Buchs über diese Konzepte informieren.

## Finanzmärkte und Machine Learning

Manche Leser sind sehr darauf erpicht, die hier vorgestellten Verfahren auf die
**Vorhersage von Wertpapierkursen (oder den Wechselkurs von Fremdwährungen usw.)** anzuwenden.

- Finanzmärkte besitzen allerdings *völlig andere statistische Eigenschaften als natürliche Phänomene* wie z.B. der Verlauf des Wetters. 
- Zu versuchen, Finanzmärkte durch Machine Learning zu übertreffen, wenn nur öffentlich zugängliche Daten verfügbar sind, ist ein schwieriges Unterfangen. 
- Sie werden aller Wahrscheinlichkeit nach nur Zeit und Geld verschwenden und kein Ergebnis vorweisen können.

Sie dürfen auch nicht vergessen, dass das vorangegangene Verhalten der Finanzmärkte kein guter Indikator für die zu erwartenden Renditen und Erträge ist – beim Autofahren ausschliesslich in den Rückspiegel zu sehen, ist ebenfalls keine
vernünftige Fahrweise. Andererseits ist Machine Learning durchaus auf Datenmengen
anwendbar, bei denen die Vergangenheit tatsächlich ein guter Indikator
für die Zukunft ist.